# Twitter Beziehungen im Parlament ermitteln

In [4]:
### import libraries
import pandas as pd
import numpy as np
import math
import requests
from bs4 import BeautifulSoup as bs
import time

---

In [142]:
### create function to find twitter-account with google query
def google_results(name, party):
    
    url = 'https://google.com/search?q={}+{}+twitter'.format(name.replace(" ", "+"), party)
    # ...wait for it....
    time.sleep(1)
    
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    twitter_handle = soup.cite.text
    
    return twitter_handle

---

In [133]:
### Download information from Wikipedia and combine in new DataFrame
nationalrat = pd.read_html("https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Schweizer_Nationalrats_in_der_50._Legislaturperiode")[0]
staenderat = pd.read_html("https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Schweizer_St%C3%A4nderats_in_der_50._Legislaturperiode")[0]

In [134]:
nationalrat = nationalrat[["Name", "Partei", "Kanton"]]
nationalrat["Rat"] = "nr"

In [135]:
staenderat = staenderat[["Name", "Partei", "Kanton"]]
staenderat["Rat"] = "sr"

In [138]:
parlament = nationalrat.append(staenderat).reset_index().drop(columns=["index"])

---

In [143]:
### Use Google to find Twitter-Page
twitter_info_list = []
for idx, row in parlament.iterrows():
    twitter_info_list.append([row["Name"], row["Partei"], google_results(row["Name"], row["Partei"])])

In [144]:
twitter_info = pd.DataFrame(twitter_info_list)
twitter_info.columns = ["name", "party", "url"]

In [146]:
### Extract Twitter-Handle from URL
def extract_handle(url):
    if (url.find("twitter") > -1):
        if len(url.split('/')) == 4:
            return(url.split('/')[3].split('?')[0])
        

In [147]:
twitter_info["twitter_handle"] = twitter_info.url.apply(extract_handle)

In [149]:
### export for manual corrections
twitter_info.to_csv("data/twitter_accounts_2019.csv")

In [14]:
### import corrected data
twitter_info_edited = pd.read_csv("data/twitter_accounts_2019_edited.csv", index_col=0)

---

To be save: Compare information from Wikipedia with official list of politicians from parlament.ch

In case of differences: Edit csv and reimport

In [260]:
parlament_2 = pd.read_excel("data/mitgliederlist-pc.xls")
names = parlament_2[["Vorname", "Name", "Fraktion"]]
names["Full_Name"] = names["Vorname"] + " " + names["Name"]

WARNING *** file size (139136) not 512 + multiple of sector size (512)


In [281]:
print(set(names["Full_Name"]) - set(twitter_info_edited.name))

{'Andrea Gmür-Schönenberger',
 'Andrea Martina Geissbühler',
 'Gregor Rutz',
 'Jean-Pierre Grin',
 'Matthias Samuel Jauslin',
 'Mike Egger',
 'Niklaus-Samuel Gugger',
 'Pascale Bruderer Wyss',
 'Philipp Matthias Bregy',
 'Ruth Humbel',
 'Sandra Sollberger'}

In [282]:
print(set(twitter_info_edited.name) - set(names["Full_Name"]))

{'Andrea Geissbühler',
 'Andrea Gmür',
 'Gregor A. Rutz',
 'Jean-Pierre Grin-Hofmann',
 'Matthias Jauslin',
 'Nik Gugger',
 'Pascale Bruderer',
 'Rebecca Ruiz',
 'Ruth Humbel Näf',
 'Sandra Sollberger-Muff'}

---

In [15]:
### Keep only users with Twitter-Account
twitter_users = twitter_info_edited[(twitter_info_edited['twitter_handle'].notnull())]

In [16]:
### Clean and save the data
twitter_users = twitter_users.drop(columns = ["edited"])
twitter_users.to_pickle("data/twitter_users.pkl")

---

In [1]:
import tweepy

In [191]:
### Initialize Twitter-API
consumer_key = "QiFcycH0Dkh2qUMKiEz6KagWy"
consumer_secret = "RJXtTYRe0lI47SXvTVcQZxnihOM7CQW88O9BptgWRWVOGQIlKU"
access_token = "20507198-nXLoFruaOgKrQZiorgJ8SJcISI9DLu1DETgHiiAWq"
access_token_secret = "YO2VQm3frJpVSCcjXFSGynQHeBMbKi3kjj1IkvKcXE3XA"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [198]:
api = tweepy.API(auth)

In [295]:
### Collect User-Ids of Twitter-Users 
def get_twitter_id(username):
    user = api.get_user(username)
    return user._json["id"]
    
twitter_users["user_id"] = twitter_users.twitter_handle.apply(get_twitter_id)
twitter_users["user_id"] = twitter_users["user_id"].astype(str)

In [350]:
### Create new DF to store Network-Data
follower_list = pd.DataFrame(columns=["Name", "Party", "Handle", "Twitter_ID", "Follower_ID"])

In [452]:
### Get Followers of each user and keep only the ones we're interested in
for idx, row in twitter_users[].iterrows():
    ids = []
    for page in tweepy.Cursor(api.followers_ids, screen_name=row.twitter_handle).pages():
        ids.extend(page)
        time.sleep(60)
    for follower in ids:
        if ((twitter_users['user_id'].astype(str) == str(follower)).any()):
            follower_list = follower_list.append(pd.DataFrame({"Name": row.name,
                                                               "Party": row.party,
                                                               "Handle": row.twitter_handle,
                                                               "Twitter_ID": row.user_id,
                                                               "Follower_ID": str(follower)
                                                              }, index=[0]), ignore_index=True)
    print(row.twitter_handle)

enussbi
paganini_nr
robipanta
gerhardpfister
valerie_piller
hpportmann
quadrantir
lorenzquadri
fregazzi
lukasreimann
maximilian_rei
mathiasreynard
natalierickli
kathyriklin
jfrime
roduitbenjamin
marcoromanoppd
regularytz
wernersalzmann
regine_sauter
schenkersilvia
schneedani67
elisabeth_s_s
uschneiderschue
seilergraf
h_siegenthaler
carlosommaruga
luzistamm
babasteinemann
adelethorens
mtoengi
manueltornare
alinetrede
m_tuena
hansuelivogt
cwasi
fwasserfallen
laurentwehrli
cedricwermuth
adrianwuethrich
zac1967
pirminbischof
pascalebruderer
andreacaroniar
raphaelcomtece
robertcramer_ge
engler_stefan
e_ettlin
JR_Fournier
olifrancais
graberkonrad
ruedinoser
b_haeberli
werner_hoesli
ClaudeJaniak
danieljositsch
christianlevrat
lombardi1956
w_luginbuehl
damian_mueller_
paulrechsteiner
riederbeat
beat_vonlanthen
zanettiroberto
mike_egger
pmbregy


In [453]:
### Save data
follower_list.to_pickle("data/follower_all.pkl")

---

In [434]:
### ... and read the data
follower_list = pd.read_pickle("data/follower_all.pkl")

In [455]:
### Check again for unwanted accounts
follower_list_red = follower_list[follower_list['Follower_ID'].isin(twitter_users['user_id'].astype(str))]

In [456]:
### Merge collected information into new DF based on Twitter-ID
followers_complete = pd.merge(follower_list_red,
                              twitter_users[["user_id", "name", "twitter_handle", "party"]],
                              left_on="Follower_ID",
                              right_on="user_id",
                              how="left")

In [457]:
# Re-Order and Clean DataFrame
followers_complete = pd.merge(followers_complete,
                             twitter_users[["name", "twitter_handle"]],
                             left_on="Handle",
                             right_on="twitter_handle",
                             how="left")

In [458]:
followers_complete = followers_complete[["name_y", "Name", "Party", "Handle", "Twitter_ID", "Follower_ID", "user_id", "name_x", "twitter_handle_x", "party", "twitter_handle_y"]]


In [459]:
followers_complete = followers_complete.drop(columns=["Name", "twitter_handle_y", "user_id"])

In [460]:
followers_complete.columns = ["user_Name", "user_Party", "user_Handle", "user_ID", "follower_ID", "follower_Name", "follower_Handle", "follower_Party"]

In [554]:
## Change parties to parliamentary groups
followers_complete.user_Party[followers_complete.user_Party == "Lega"] = "SVP"
followers_complete.follower_Party[followers_complete.follower_Party == "Lega"] = "SVP"

followers_complete.user_Party[followers_complete.user_Party == "EVP"] = "CVP"
followers_complete.follower_Party[followers_complete.follower_Party == "EVP"] = "CVP"

followers_complete.user_Party[followers_complete.user_Party == "EVP"] = "CVP"
followers_complete.follower_Party[followers_complete.follower_Party == "EVP"] = "CVP"

/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/sco/miniconda3/lib/python3.7/site-p

---

In [461]:
### Experiment: create scale of parties an look at distances
def party_scale(party):
    scale = {
        "sp": 1,
        "gps": 2,
        "evp": 3,
        "glp": 4,
        "cvp": 5,
        "bdp": 6,
        "fdp": 7,
        "lega": 8,
        "mcg": 9,
        "svp": 10
    }
    return scale[party.lower()]

In [462]:
followers_complete["user_Party_Scale"] = followers_complete["user_Party"].apply(party_scale)
followers_complete["follower_Party_Scale"] = followers_complete["follower_Party"].apply(party_scale)

In [463]:
followers_complete["Scale_Diff"] = abs(followers_complete["user_Party_Scale"] - followers_complete["follower_Party_Scale"])

---

In [697]:
### Save Data for analysis
followers_complete.to_pickle("data/followers_complete.pkl")

---